Import Packeges

In [1]:
import tensorflow as tf
from glob import glob

import mne

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
# from tensorflow.keras.metrics import Accuracy, Precision, Recall, F1Score, AUC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler

Read Input Fiels

In [2]:
#read all file
all_files_path=glob('test/*.edf')
print(len(all_files_path))

18


Separate labeled data
H= Health control
M= MDD


In [3]:
healthy_file_path=[i for i in all_files_path if  'H' in i.split('\\')[1]]
patient_file_path=[i for i in all_files_path if  'M' in i.split('\\')[1]]

print(len(healthy_file_path),len(patient_file_path))

9 9


Plot a data

In [ ]:
%%capture
H_data_for_plt = mne.io.read_raw_edf(healthy_file_path[1], preload=True)
H_data_for_plt.pick_types(meg=False, eeg=True, eog=False, ecg=False) # Selecting EEG, EOG and ECG channels
# Select a specific channel
channel_to_keep = ['EEG Fp1-LE', 'EEG F3-LE', 'EEG C3-LE', 'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 'EEG T3-LE', 'EEG T5-LE', 'EEG Fz-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG C4-LE', 'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE', 'EEG T4-LE', 'EEG T6-LE', 'EEG Cz-LE', 'EEG Pz-LE']  
# Replace with the name of the channel you want to keep
H_data_for_plt.pick_channels(channel_to_keep)
H_data_for_plt.set_eeg_reference()
H_data_for_plt.filter(l_freq=30,h_freq=100)#1-4=delta, 4-8=theta, 8-12=alpha, 12-30=beta, 30-100=gamma
# H_data_for_plt=mne.make_fixed_length_epochs(H_data_for_plt,duration=15,overlap=1)
# H_data_for_plt=H_data_for_plt.get_data()

In [ ]:
# H_data_for_plt.plot()

Read data

select gamma wave

15 sec duration epochs

Standardize features by removing the mean and scaling to unit variance.
The standard score of a sample x is calculated as:
z = (x - u) / s

In [4]:
def read_data(file_path):
    raw = mne.io.read_raw_edf(file_path, preload=True)
    raw.pick_types(meg=False, eeg=True, eog=False, ecg=False) # Selecting EEG, EOG and ECG channels
    # Select a specific channel
    channel_to_keep = ['EEG Fp1-LE', 'EEG F3-LE', 'EEG C3-LE', 'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 'EEG T3-LE', 'EEG T5-LE', 'EEG Fz-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG C4-LE', 'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE', 'EEG T4-LE', 'EEG T6-LE', 'EEG Cz-LE', 'EEG Pz-LE']  
    # Replace with the name of the channel you want to keep
    raw.pick_channels(channel_to_keep)
    raw.set_eeg_reference()
    raw.filter(l_freq=30,h_freq=100)#1-4=delta, 4-8=theta, 8-12=alpha, 12-30=beta, 30-100=gamma
    epochs=mne.make_fixed_length_epochs(raw,duration=15,overlap=1)
    epochs=epochs.get_data()
    scaler = StandardScaler()
    data = scaler.fit_transform(epochs.reshape(-1,epochs.shape[-1])).reshape(epochs.shape)
    return data #trials,channel,length

In [5]:
%%capture
control_epochs_array=[read_data(subject) for subject in healthy_file_path]
patients_epochs_array=[read_data(subject) for subject in patient_file_path]

In [6]:
control_epochs_labels=[len(i)*[0] for i in control_epochs_array]
patients_epochs_labels=[len(i)*[1] for i in patients_epochs_array]
print(len(control_epochs_labels),len(patients_epochs_labels))

9 9


In [7]:
data_list=control_epochs_array+patients_epochs_array
label_list=control_epochs_labels+patients_epochs_labels
groups_list=[[i]*len(j) for i, j in enumerate(data_list)]
print(len(data_list),len(label_list),len(groups_list))

18 18 18


In [8]:
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(groups_list)
print(data_array.shape,label_array.shape,group_array.shape)

(543, 19, 3840) (543,) (543,)


In [9]:
data_array=np.moveaxis(data_array,1,2)

In [10]:
def cnnmodel():
    clear_session()
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(3840,19)))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#8
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())#10
    model.add(Dense(1,activation='sigmoid'))#11
    
    model.compile('adam',loss='binary_crossentropy',metrics=['Accuracy', 'Precision', 'Recall','AUC'])
    return model

model=cnnmodel()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 3838, 5)           290       
                                                                 
 batch_normalization (Batch  (None, 3838, 5)           20        
 Normalization)                                                  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 3838, 5)           0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 1919, 5)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 1917, 5)           80        
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 1917, 5)           0

In [11]:
gkf=GroupKFold(n_splits=10) #K-fold iterator with 10 fold
accuracy=[]
precision = []
recall=[]
AUC=[]
F1Score=[]
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=cnnmodel()
    model.fit(train_features,train_labels,epochs=15,batch_size=32,validation_data=(val_features,val_labels))
    evaluation_results = model.evaluate(val_features,val_labels)
    accuracy.append(evaluation_results[1])
    precision.append(evaluation_results[2])
    recall.append(evaluation_results[3])
    # F1Score.append(evaluation_results[4])
    AUC.append(evaluation_results[4])

Epoch 1/15
16/16 [==============================] - 5s 124ms/step - loss: 0.6870 - Accuracy: 0.5171 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.6172 - val_loss: 0.7127 - val_Accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.0000e+00
Epoch 2/15
16/16 [==============================] - 1s 73ms/step - loss: 0.6656 - Accuracy: 0.5171 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.8150 - val_loss: 0.7347 - val_Accuracy: 0.0217 - val_precision: 1.0000 - val_recall: 0.0217 - val_auc: 0.0000e+00
Epoch 3/15
16/16 [==============================] - 1s 73ms/step - loss: 0.6266 - Accuracy: 0.5654 - precision: 1.0000 - recall: 0.1000 - auc: 0.8813 - val_loss: 0.7592 - val_Accuracy: 0.2391 - val_precision: 1.0000 - val_recall: 0.2391 - val_auc: 0.0000e+00
Epoch 4/15
16/16 [==============================] - 1s 71ms/step - loss: 0.5523 - Accuracy: 0.7364 - precision: 0.9360 - recall: 0.4875 - auc: 0.8995 - val_loss: 0.7033 - val_Accuracy: 0.5435 - 

In [12]:
print(np.mean(accuracy),np.mean(precision),np.mean(recall),np.mean(AUC))

0.8124156773090363 0.6524999976158142 0.6457142889499664 0.3887188255786896


In [13]:
model.save("gamma_15s_15epoch_32batch.h5")

c:\Users\rahat\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
